In [5]:
#01_cleandata.ipynb
    #this script is used to re-structure, clean and conduct initial data checks of data
#Created: 07.09.2024
#By: Alessia Kettlitz

#importing libraries
import pandas as pd, numpy as np, matplotlib as plt, seaborn as sns, pyprojroot as root

from pyprojroot.here import here #importing here function for relative file paths

from joblib import dump, load #for saving dataframes


In [6]:
#importing data
data_raw = pd.read_csv(here('Raw_Data/National_Vital_Statistics_System__NVSS__-_National_Cardiovascular_Disease_Surveillance_Data_20240709.csv'))

C:\Users\Alessia\AppData\Local\Temp\ipykernel_7768\132631249.py:2: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  data_raw = pd.read_csv(here('Raw_Data/National_Vital_Statistics_System__NVSS__-_National_Cardiovascular_Disease_Surveillance_Data_20240709.csv'))


Initial Data Exploration

In [8]:
#printing head of data
data_raw.head(25)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174720 entries, 0 to 174719
Data columns (total 30 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   RowId                       174720 non-null  object 
 1   YearStart                   174720 non-null  int64  
 2   LocationAbbr                174720 non-null  object 
 3   LocationDesc                174720 non-null  object 
 4   DataSource                  174720 non-null  object 
 5   PriorityArea1               43680 non-null   object 
 6   PriorityArea2               0 non-null       float64
 7   PriorityArea3               43680 non-null   object 
 8   PriorityArea4               0 non-null       float64
 9   Class                       174720 non-null  object 
 10  Topic                       174720 non-null  object 
 11  Question                    174720 non-null  object 
 12  Data_Value_Type             174720 non-null  object 
 13  Data_Value_Uni

In [9]:
#checking info of data
data_raw.info()
    #174,720 observations
    #30 variables

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174720 entries, 0 to 174719
Data columns (total 30 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   RowId                       174720 non-null  object 
 1   YearStart                   174720 non-null  int64  
 2   LocationAbbr                174720 non-null  object 
 3   LocationDesc                174720 non-null  object 
 4   DataSource                  174720 non-null  object 
 5   PriorityArea1               43680 non-null   object 
 6   PriorityArea2               0 non-null       float64
 7   PriorityArea3               43680 non-null   object 
 8   PriorityArea4               0 non-null       float64
 9   Class                       174720 non-null  object 
 10  Topic                       174720 non-null  object 
 11  Question                    174720 non-null  object 
 12  Data_Value_Type             174720 non-null  object 
 13  Data_Value_Uni

In [10]:
#basic stats for all variables in dataframe
data_raw.describe(include='all')

,RowId,YearStart,LocationAbbr,LocationDesc,DataSource,PriorityArea1,PriorityArea2,PriorityArea3,PriorityArea4,Class,...,Break_Out_Category,Break_Out,ClassId,TopicId,QuestionId,Data_Value_TypeID,BreakOutCategoryId,BreakOutId,LocationId,Geolocation
count,174720,174720.000000,174720,174720,174720,43680,0.0,43680,0.0,174720,...,174720,174720,174720,174720,174720,174720,174720,174720,174720.000000,171360
unique,174720,NaN,52,52,1,1,NaN,1,NaN,1,...,4,13,1,6,8,2,4,13,NaN,51
top,NVSS~2020~59~NV008~GEN01~Age-Standardized,NaN,AL,Alabama,NVSS,none,NaN,Healthy People 2030,NaN,Cardiovascular Diseases,...,Race,Overall,C1,T6,NV001,Crude,BOC04,OVR01,NaN,POINT (-86.63186076 32.84057112)
freq,1,NaN,3360,3360,174720,43680,NaN,43680,NaN,174720,...,69888,17472,174720,65520,21840,113568,69888,17472,NaN,3360
mean,NaN,2010.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.538462,NaN
std,NaN,6.055318,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.064174,NaN
min,NaN,2000.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN
25%,NaN,2005.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.750000,NaN
50%,NaN,2010.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.500000,NaN
75%,NaN,2015.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.500000,NaN


In [12]:
#filtering dataset to where RowId contains string 'Age-Standardized' - we will subset just to standardized estimates by age
data_filter = data_raw[(data_raw['RowId'].str.contains('Age-Standardized')) & (data_raw['BreakOutId'] == 'OVR01')]

data_filter
#this reduces to 8736 rows - more manageable for this project for now!

,RowId,YearStart,LocationAbbr,LocationDesc,DataSource,PriorityArea1,PriorityArea2,PriorityArea3,PriorityArea4,Class,...,Break_Out_Category,Break_Out,ClassId,TopicId,QuestionId,Data_Value_TypeID,BreakOutCategoryId,BreakOutId,LocationId,Geolocation
0,NVSS~2000~1~NV001~OVR01~Age-Standardized,2000,AL,Alabama,NVSS,NaN,NaN,NaN,NaN,Cardiovascular Diseases,...,Overall,Overall,C1,T1,NV001,AgeStdz,BOC01,OVR01,1,POINT (-86.63186076 32.84057112)
21,NVSS~2000~2~NV001~OVR01~Age-Standardized,2000,AK,Alaska,NVSS,NaN,NaN,NaN,NaN,Cardiovascular Diseases,...,Overall,Overall,C1,T1,NV001,AgeStdz,BOC01,OVR01,2,POINT (-147.722059 64.84507996)
40,NVSS~2000~4~NV001~OVR01~Age-Standardized,2000,AZ,Arizona,NVSS,NaN,NaN,NaN,NaN,Cardiovascular Diseases,...,Overall,Overall,C1,T1,NV001,AgeStdz,BOC01,OVR01,4,POINT (-111.7638113 34.86597028)
61,NVSS~2000~5~NV001~OVR01~Age-Standardized,2000,AR,Arkansas,NVSS,NaN,NaN,NaN,NaN,Cardiovascular Diseases,...,Overall,Overall,C1,T1,NV001,AgeStdz,BOC01,OVR01,5,POINT (-92.27449074 34.74865012)
80,NVSS~2000~6~NV001~OVR01~Age-Standardized,2000,CA,California,NVSS,NaN,NaN,NaN,NaN,Cardiovascular Diseases,...,Overall,Overall,C1,T1,NV001,AgeStdz,BOC01,OVR01,6,POINT (-120.9999995 37.63864012)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174621,NVSS~2020~53~NV008~OVR01~Age-Standardized,2020,WA,Washington,NVSS,NaN,NaN,NaN,NaN,Cardiovascular Diseases,...,Overall,Overall,C1,T6,NV008,AgeStdz,BOC01,OVR01,53,POINT (-120.4700108 47.52227863)
174641,NVSS~2020~54~NV008~OVR01~Age-Standardized,2020,WV,West Virginia,NVSS,NaN,NaN,NaN,NaN,Cardiovascular Diseases,...,Overall,Overall,C1,T6,NV008,AgeStdz,BOC01,OVR01,54,POINT (-80.71264013 38.6655102)
174660,NVSS~2020~55~NV008~OVR01~Age-Standardized,2020,WI,Wisconsin,NVSS,NaN,NaN,NaN,NaN,Cardiovascular Diseases,...,Overall,Overall,C1,T6,NV008,AgeStdz,BOC01,OVR01,55,POINT (-89.81637074 44.39319117)
174681,NVSS~2020~56~NV008~OVR01~Age-Standardized,2020,WY,Wyoming,NVSS,NaN,NaN,NaN,NaN,Cardiovascular Diseases,...,Overall,Overall,C1,T6,NV008,AgeStdz,BOC01,OVR01,56,POINT (-108.1098304 43.23554134)


In [13]:
#drop columns that will not be used
columns_to_drop = ['DataSource', 'PriorityArea1', 'PriorityArea2', 'PriorityArea3', 'PriorityArea4', 'Class', 'Break_Out_Category', 'Break_Out', 'ClassId', 'Data_Value_TypeID', 'BreakOutCategoryId', 'BreakOutId']
data_filter.drop(columns=columns_to_drop, inplace=True)

C:\Users\Alessia\AppData\Local\Temp\ipykernel_7768\1086600172.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filter.drop(columns=columns_to_drop, inplace=True)


In [14]:
#compute descriptive statistics grouped by 'Question'
stats_by_question = data_filter.groupby(['YearStart', 'LocationAbbr', 'Question'])['Data_Value'].describe()

stats_by_question

count  \
YearStart LocationAbbr Question                                                    
2000      AK           Acute myocardial infarction (heart attack) mort...    1.0   
                       Cerebrovascular disease (stroke) mortality rate...    1.0   
                       Coronary heart disease mortality rate among US ...    1.0   
                       Diseases of the heart (heart disease) mortality...    1.0   
                       Heart failure mortality rate among US adults (1...    1.0   
...                                                                          ...   
2020      WY           Diseases of the heart (heart disease) mortality...    1.0   
                       Heart failure mortality rate among US adults (1...    1.0   
                       Hemorrhagic stroke mortality rate among US adul...    1.0   
                       Ischemic stroke mortality rate among US adults ...    1.0   
                       Major cardiovascular disease mortality rate amo...    1.0   

                                                                            mean  \
YearStart LocationAbbr Question                                                    
2000      AK           Acute myocardial infarction (heart attack) mort...   46.1   
                       Cerebrovascular disease (stroke) mortality rate...   65.0   
                       Coronary heart disease mortality rate among US ...  149.4   
                       Diseases of the heart (heart disease) mortality...  221.2   
                       Heart failure mortality rate among US adults (1...   18.5   
...                                                                          ...   
2020      WY           Diseases of the heart (heart disease) mortality...  197.5   
                       Heart failure mortality rate among US adults (1...   23.0   
                       Hemorrhagic stroke mortality rate among US adul...   11.0   
                       Ischemic stroke mortality rate among US adults ...   19.4   
                       Major cardiovascular disease mortality rate amo...  249.6   

                                                                           std  \
YearStart LocationAbbr Question                                                  
2000      AK           Acute myocardial infarction (heart attack) mort...  NaN   
                       Cerebrovascular disease (stroke) mortality rate...  NaN   
                       Coronary heart disease mortality rate among US ...  NaN   
                       Diseases of the heart (heart disease) mortality...  NaN   
                       Heart failure mortality rate among US adults (1...  NaN   
...                                                                        ...   
2020      WY           Diseases of the heart (heart disease) mortality...  NaN   
                       Heart failure mortality rate among US adults (1...  NaN   
                       Hemorrhagic stroke mortality rate among US adul...  NaN   
                       Ischemic stroke mortality rate among US adults ...  NaN   
                       Major cardiovascular disease mortality rate amo...  NaN   

                                                                             min  \
YearStart LocationAbbr Question                                                    
2000      AK           Acute myocardial infarction (heart attack) mort...   46.1   
                       Cerebrovascular disease (stroke) mortality rate...   65.0   
                       Coronary heart disease mortality rate among US ...  149.4   
                       Diseases of the heart (heart disease) mortality...  221.2   
                       Heart failure mortality rate among US adults (1...   18.5   
...                                                                          ...   
2020      WY           Diseases of the heart (heart disease) mortality...  197.5   
                       Heart failure mortality rate among US adult

In [15]:
#export dataframe to joblib file
dump(data_filter, here('Clean_Data/data.joblib'))

['c:\\Users\\Alessia\\Documents\\Personal Data Projects\\Python Example\\cardiovascular-surveillance\\Clean_Data\\data.joblib']